In [26]:
# import packages
import pandas as pd

In [27]:
# import data
firestore_data_label_path = pd.read_csv("../../data/label_path/export_table_path_label_4.4.2021.csv",
                                        skiprows=[1],
                                        dtype={"logo": object, "path_0": bool, "path_1": bool, "path_2": bool,
                                               "path_3": bool, "path_4": bool,
                                               "path_5": bool, "path_6": bool, "path_7": bool, "alias": object})

In [28]:
# extract id
data_label_path_id = firestore_data_label_path.assign(
    logo_id=firestore_data_label_path.logo.str.extract("(\d+)")).astype({"logo_id": 'int64'})
data_label_path_id_melt = pd.melt(data_label_path_id, id_vars=["logo", "alias", "logo_id"],
                                  value_vars=["path_0", "path_1", "path_2", "path_3", "path_4", "path_5", "path_6",
                                              "path_7"],
                                  var_name="path", value_name="animate")
data_label_path_id_melt_type = data_label_path_id_melt.assign(
    order_id=data_label_path_id_melt.path.str.extract("(\d+)")).astype({"order_id": 'int64'})

In [29]:
data_label_path_id_melt_type

,logo,alias,logo_id,path,animate,order_id
0,path/logo_303_path_full.svg,Tim,303,path_0,True,0
1,path/logo_283_path_full.svg,Sarah,283,path_0,True,0
2,path/logo_162_path_full.svg,TestJani,162,path_0,False,0
3,path/logo_3_path_full.svg,Jonas,3,path_0,False,0
4,path/logo_274_path_full.svg,Löschen,274,path_0,False,0
...,...,...,...,...,...,...
33187,path/logo_413_path_full.svg,Jakob,413,path_7,False,7
33188,path/logo_154_path_full.svg,Niklas,154,path_7,False,7
33189,path/logo_12_path_full.svg,rebecca,12,path_7,False,7
33190,path/logo_216_path_full.svg,Jonathan,216,path_7,True,7


In [39]:
# create matching dataframe
data_matching_filename_new = pd.read_csv("../../data/label_path/label_matching_scraped_svgs.csv")
data_matching_filename_old = pd.read_csv("../../data/label_path/label_matching.csv")

data_matching_filename_new_clean = data_matching_filename_new.assign(
    filename = data_matching_filename_new.logo_id,
    logo_id = data_matching_filename_new.logo_id.str.extract("(\d+)"))

data_matching_filename_id = pd.concat([data_matching_filename_old,
                                       data_matching_filename_new_clean]).astype({"logo_id": 'int64'})

In [40]:
data_matching_filename_id

,filename,logo_id,order_id,animation_id
0,L_7sins_01,0,0,5
1,L_7sins_01,0,1,0
2,L_7sins_01,0,2,2
3,L_7sins_01,0,3,4
4,L_7sins_01,0,4,3
...,...,...,...,...
1910,logo_436,436,3,12
1911,logo_436,436,4,18
1912,logo_436,436,5,15
1913,logo_436,436,6,13


In [41]:
# merge data
label_path_merged = pd.merge(data_label_path_id_melt_type,
                             data_matching_filename_id, how='left', on=["logo_id", "order_id"])

In [48]:
label_path_merged.groupby("alias").size()

alias
FSF LARI           8
Jakob           3512
Jan B.          1968
Jani            1536
Jonas            176
Jonathan        3632
Julia           1536
Kikipu          1536
Lena            1536
Löschen         1992
Niklas          3312
Peter             56
Ramo            1592
Rebecca            8
Sarah           1960
Sarah_240321    1536
TIm               16
Team sari         56
TestJani        1568
Tim             2376
Tim_Harry       1536
Vorhang           16
rebecca         1536
test              32
dtype: int64

In [49]:
# filter data
label_path_merged_filter = label_path_merged[
    label_path_merged.alias.isin(["Jani", "Jakob", "Jonathan", "Julia", "Kikipu", "Lena",
                                  "Niklas", "Ramo", "rebecca", "Sarah_240321", "Tim_Harry",
                                  "Jan B.","Sarah", "Tim"])].dropna()

In [50]:
label_path_merged_filter

,logo,alias,logo_id,path,animate,order_id,filename,animation_id
0,path/logo_303_path_full.svg,Tim,303,path_0,True,0,logo_303,12.0
1,path/logo_283_path_full.svg,Sarah,283,path_0,True,0,logo_283,22.0
5,path/logo_32_path_full.svg,Tim,32,path_0,True,0,L_Braumann_05,134.0
8,path/logo_119_path_full.svg,rebecca,119,path_0,False,0,L_MTM_03,0.0
9,path/logo_37_path_full.svg,Niklas,37,path_0,True,0,L_compassus_2,0.0
...,...,...,...,...,...,...,...,...
33187,path/logo_413_path_full.svg,Jakob,413,path_7,False,7,logo_413,24.0
33188,path/logo_154_path_full.svg,Niklas,154,path_7,False,7,L_Prime_People_3,7.0
33189,path/logo_12_path_full.svg,rebecca,12,path_7,False,7,L_Beck_Bau_06_1,1.0
33190,path/logo_216_path_full.svg,Jonathan,216,path_7,True,7,logo_216,10.0


In [51]:
# aggregate data
path_label_final = label_path_merged_filter.groupby(by=["filename", "logo_id", "order_id", "animation_id"]).mean(
    "animate")

In [52]:
# aggregate data
path_label_final

animate
filename   logo_id order_id animation_id          
L_7sins_01 0       0        5.0           0.578947
                   1        0.0           0.526316
                   2        2.0           0.526316
                   3        4.0           0.684211
                   4        3.0           0.526316
...                                            ...
logo_436   436     3        12.0          0.200000
                   4        18.0          0.200000
                   5        15.0          0.200000
                   6        13.0          0.200000
                   7        17.0          0.200000

[3441 rows x 1 columns]

In [ ]:

# export data
path_label_final.to_csv("../../data/label_path/animation_path_label.csv")

In [ ]:

# extract id
data_label_path_id = firestore_data_label_path.assign(
    logo_id=firestore_data_label_path.logo.str.extract("(\d+)")).astype({"logo_id": 'int64'})
data_label_path_id_melt = pd.melt(data_label_path_id, id_vars=["logo", "alias", "logo_id"],
                                  value_vars=["path_0", "path_1", "path_2", "path_3", "path_4", "path_5", "path_6",
                                              "path_7"],
                                  var_name="path", value_name="animate")
data_label_path_id_melt_type = data_label_path_id_melt.assign(
    order_id=data_label_path_id_melt.path.str.extract("(\d+)")).astype({"order_id": 'int64'})

# merge data
label_path_merged = pd.merge(data_label_path_id_melt_type,
                             data_matching_filename_id, how='left', on=['logo_id', "order_id"])

# filter data
label_path_merged_filter = label_path_merged[
    label_path_merged.alias.isin(["Jani", "Jakob", "Jonathan", "Julia", "Kikipu", "Lena",
                                  "Niklas", "Ramo", "rebecca", "Sarah_240321", "Tim_Harry"])].dropna()
path_label_final = label_path_merged_filter.groupby(by=["filename", "logo_id", "order_id", "animation_id"]).mean(
    "animate")

# export data
path_label_final.to_csv("../../data/label_path/animation_path_label.csv")




In [9]:
data_label_path_id[data_label_path_id.logo_id > 192].groupby(by=["alias"]).size()

alias
Jakob       246
Jan B.      245
Jonathan    252
Löschen     248
Ramo          7
Sarah       244
Tim         244
dtype: int64